<a href="https://colab.research.google.com/github/JSJeong-me/RPA-2023/blob/main/%EC%98%88%EC%B8%A1%EB%B6%84%EC%84%9D/0520_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_excel("lotto1068.xlsx")

In [ ]:
df.head()

In [4]:
df.tail()

,회차,번호1,번호2,번호3,번호4,번호5,번호6
1062,1063,3,6,22,23,24,38
1063,1064,3,6,9,18,22,35
1064,1065,6,11,16,18,21,32
1065,1066,6,9,12,14,31,32
1066,1068,0,0,0,0,0,0


In [ ]:
df.info()

In [5]:
df.drop(['회차'], axis=1, inplace=True)

In [ ]:
df.describe()

In [ ]:
# df.drop(['Game', 'Date'], axis=1, inplace=True)

In [ ]:
df.head()

In [6]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [7]:
transformed_df.head()

,0,1,2,3,4,5
0,0.618633,1.438877,1.209528,0.887531,0.606351,0.098715
1,0.431535,-0.011144,0.139961,-0.160320,-0.114273,0.466357
2,0.805731,0.423862,-0.127431,-0.684246,-0.834898,-1.555672
3,1.367025,2.018885,1.343224,0.625568,1.038726,0.466357
4,1.741221,1.583879,1.209528,1.804402,1.182851,0.466357


In [8]:
# All our games
number_of_rows = df.values.shape[0]
number_of_rows

1067

In [9]:
# Amount of games we need to take into consideration for prediction
window_length = 7
window_length 

7

In [10]:
# Balls counts
number_of_features = df.values.shape[1]
number_of_features

6

In [ ]:
X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
X

In [ ]:
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)
y

In [ ]:
X.shape

In [ ]:
y.shape

In [13]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X[0]

In [ ]:
X[1]

In [ ]:
y[0]

In [ ]:
y[1]

In [14]:
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
batch_size = 2000

In [ ]:
# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(59))
# Adding the last output layer
model.add(Dense(number_of_features))

In [ ]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

In [ ]:
model.fit(x=X, y=y, batch_size=100, epochs=10000, verbose=2)

1066모델 Load

In [15]:
from tensorflow import keras
model = keras.models.load_model("./winner_table/new_model_1066.h5")

In [16]:
to_predict = df.tail(8)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

<ipython-input-16-4b4c855b4438>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)


,번호1,번호2,번호3,번호4,번호5,번호6
1059,3,10,24,33,38,45
1060,4,24,27,35,37,45
1061,20,31,32,40,41,45
1062,3,6,22,23,24,38
1063,3,6,9,18,22,35
1064,6,11,16,18,21,32
1065,6,9,12,14,31,32


In [17]:
to_predict = np.array(to_predict)
to_predict

array([[ 3, 10, 24, 33, 38, 45],
       [ 4, 24, 27, 35, 37, 45],
       [20, 31, 32, 40, 41, 45],
       [ 3,  6, 22, 23, 24, 38],
       [ 3,  6,  9, 18, 22, 35],
       [ 6, 11, 16, 18, 21, 32],
       [ 6,  9, 12, 14, 31, 32]])

In [18]:
scaled_to_predict = scaler.transform(to_predict)

In [19]:
y_pred = model.predict(np.array([scaled_to_predict]))
print("The predicted numbers in the last lottery game are:", scaler.inverse_transform(y_pred).astype(int)[0])

1/1 [==============================] - 3s 3s/step
The predicted numbers in the last lottery game are: [ 2  8 16 19 23 37]


In [20]:
prediction = df.tail(1)
prediction = np.array(prediction)
print("The actual numbers in the last lottery game were:", prediction[0])

The actual numbers in the last lottery game were: [0 0 0 0 0 0]


### Save 모델

In [ ]:
model.save("new_model_1054_04_21_20000.h5")